In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
import os
import shutil

# Paths to your Anemia and Non_anemia folders
anemia_folder = '/content/Anemia'
non_anemia_folder = '/content/Non_anemia'

# Create a new directory to contain both classes
new_data_dir = '/content/combined/'
os.makedirs(os.path.join(new_data_dir, 'Anemia'), exist_ok=True)
os.makedirs(os.path.join(new_data_dir, 'Non_anemia'), exist_ok=True)

# Move files into respective folders in the new structure
for filename in os.listdir(anemia_folder):
    shutil.move(os.path.join(anemia_folder, filename), os.path.join(new_data_dir + 'Anemia', filename))

for filename in os.listdir(non_anemia_folder):
    shutil.move(os.path.join(non_anemia_folder, filename), os.path.join(new_data_dir + 'Non_anemia', filename))


In [11]:
data_gen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)

In [12]:
# Train generator (80% of data)
train_generator = data_gen.flow_from_directory(
    new_data_dir,  # This is the new unified directory containing the two folders
    target_size=(80, 80),  # Resize images to 80x80
    batch_size=32,
    class_mode='binary',  # Binary classification
    subset='training',  # 80% for training
    shuffle=True
)

# Validation generator (20% of data)
validation_generator = data_gen.flow_from_directory(
    new_data_dir,
    target_size=(80, 80),
    batch_size=32,
    class_mode='binary',
    subset='validation',  # 20% for validation
    shuffle=True
)

Found 569 images belonging to 2 classes.
Found 141 images belonging to 2 classes.


In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Build the CNN model
cnn_model = models.Sequential()

# First convolutional layer
cnn_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(80, 80, 3)))
cnn_model.add(layers.MaxPooling2D((2, 2)))

# Second convolutional layer
cnn_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))

# Third convolutional layer
cnn_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))

# Fourth convolutional layer
cnn_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
cnn_model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output of the convolutional layers to feed into a fully connected layer
cnn_model.add(layers.Flatten())

# Add a fully connected layer (optional if you want feature-rich output)
cnn_model.add(layers.Dense(512, activation='relu'))

# Output Layer is removed (since we're using SVM for classification)

# Summary of the model
cnn_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 78, 78, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 39, 39, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 37, 37, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 18, 18, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 6, 6, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 3, 3, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1152)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         590,336 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 831,168 (3.17 MB)

 Trainable params: 831,168 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Remove the output layer from the CNN (you can skip this if it's not added)
cnn_model.pop()

# Compile the model (without training yet)
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
import numpy as np

# Extract features for the training data
train_features = cnn_model.predict(train_generator)
train_labels = train_generator.classes  # These are the labels corresponding to the features

# Extract features for the validation data
validation_features = cnn_model.predict(validation_generator)
validation_labels = validation_generator.classes

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 130ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step 


In [16]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Standardize the features (this improves SVM performance)
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
validation_features = scaler.transform(validation_features)

# Train an SVM classifier
svm_classifier = svm.SVC(kernel='linear')
svm_classifier.fit(train_features, train_labels)

# Predict on the validation set
validation_pred = svm_classifier.predict(validation_features)

# Calculate the accuracy
accuracy = accuracy_score(validation_labels, validation_pred)
print(f"SVM Validation Accuracy: {accuracy * 100:.2f}%")

SVM Validation Accuracy: 46.10%


In [19]:
from tensorflow.keras.preprocessing import image
# Path to the new image
img_path = '/content/combined/Anemia/Image_001.png'

# Load the image
img = image.load_img(img_path, target_size=(80, 80))  # Resize the image to the same size used in training (80x80)

# Convert the image to array format (required by the CNN model)
img_array = image.img_to_array(img)

# Expand dimensions to match the batch size format (1, 80, 80, 3)
img_array = np.expand_dims(img_array, axis=0)

# Normalize the image (as done during training)
img_array /= 255.0

In [20]:
# Use the CNN to extract features from the new image
new_image_features = cnn_model.predict(img_array)

# Normalize the features using the same scaler that was used during training
new_image_features = scaler.transform(new_image_features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step


In [21]:
# Predict with the SVM classifier
prediction = svm_classifier.predict(new_image_features)

# Output the result
if prediction == 0:
    print("The image is classified as: Anemic")
else:
    print("The image is classified as: Non-anemic")

The image is classified as: Anemic
